# <center> Apply backpropagation algorithm to wheet seeds datasets</center>

In [ ]:
import numpy as np
import os

In [ ]:
dataset_dir = '/home/lidong/Datasets/ML' 

In [ ]:
def load_dataset_from_file(path):
    inputs = []
    targets = []
    with open(path) as f:
        for line in f:
            row = line.rstrip('\n').split()
            inputs.append(list(map(lambda x: float(x.strip()), row[:-1])))
            targets.append(int(row[-1].strip()))
            
    # classify by raw values
    class_values = set(targets)
    class_indexs = dict()
    for i, value in enumerate(class_values):
        class_indexs[value] = i
        
    dataset = []
    for i in range(len(targets)):
        dataset.append((inputs[i], class_indexs[targets[i]]))
    return np.array(dataset), class_indexs

In [ ]:
def normalize_dataset(dataset):
    inputs = dataset[:, 0]
    minmax = [[min(column), max(column)] for column in zip(*inputs)]
    columns = len(minmax)
    for row in inputs:
        for i in range(columns):
            row[i] = round((row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0]), 4)
    return dataset

In [ ]:
dataset, class_indexs = load_dataset_from_file(os.path.join(dataset_dir, 'seeds_dataset.txt'))
dataset = normalize_dataset(dataset)
len(dataset[0, 0])

In [ ]:
ninput = len(dataset[0, 0])
nhiden = 5
noutput = len(class_indexs)
sizes=(ninput, nhiden, noutput)

Activitation function

In [ ]:
def activite_transfer(z):
    # sigmoid
    return 1.0/(1.0+np.exp(-z))

def activite_derivative(z):
    # the derivative of the sigmoid
    return activite_transfer(z)*(1-activite_transfer(z))

In [ ]:
def forward_propagate(network, x):
    a = x
    zs = [a] # the input layer, no activite function, so 'z = a'
    activations = [a] # first for input layer
    for b, w in zip(network['biases'], network['weights']):
        z = np.dot(w, a) + b
        zs.append(z)
        a = activite_transfer(z)
        activations.append(a)
    return zs, activations

Backward propagate:
$$ 
\begin{align*} 
\delta^L &= (a^L-y) \odot \sigma'(z^L) \\
\delta^l &= ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l) \\
\dfrac{\partial C}{\partial w^l_{jk}} &= a^{l-1}_k \delta^l_j \\
\dfrac{\partial C}{\partial w^l} &= \delta^l (a^{l-1})^T \\
\dfrac{\partial C}{\partial b^l_j} &= \delta^l_j \\
\end{align*}
$$

In [ ]:
def backward_propagate(network, y, zs, activations):
    # init nabla(delta) values of weights and biases for all layers except input layer
    nabla_b = [np.zeros(b.shape) for b in network['biases']]
    nabla_w = [np.zeros(w.shape) for w in network['weights']]
    
    nlayers = network['nlayers']
    for l in reversed(range(nlayers)):
        if l == nlayers-1:
            delta = (activations[l] - y) * activite_derivative(zs[l])
        else:
            delta = np.dot(network['weights'][l-1].transpose(), delta) * activite_derivative(zs[l])
        nabla_b[l] = delta
        nabla_w[l] = np.dot(delta, activations[l-1].transpose())
        
    return (nabla_b, nabla_w)

In [ ]:
class NeuralNetwork(object):
    def __init__(self, dataset, sizes):
        self.dataset = dataset
        self.sizes = sizes
        self.network = self.initialize_network(sizes)
        # print(self.network)
        
    @staticmethod
    def initialize_network(sizes):
        network = dict()
        network['nlayers'] = len(sizes)
        network["biases"] = [np.random.randn(y, 1) for y in sizes[1:]]
        network["weights"] = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        return network
        
    def train(self, batch_size, eta, epochs):
        # SGD stochastic gradient descent
        N = len(dataset)
        for epoch in range(epochs):
            np.random.shuffle(self.dataset)
            batches = [self.dataset[k:k+batch_size] for k in range(0, N, batch_size)]
            for batch in batches:
                for x, y in batch:
                    # 1. feed forward
                    zs, activations = forward_propagate(self.network, x)
                    # 2. back propagation
                    backward_propagate(self.network, y, zs, activations)
                    # 3. update weights and biases
                    break
                break
            return


In [ ]:
network = NeuralNetwork(dataset, sizes)
network.train(50, 0.1, 100)